In [1]:
# Import modules.
import pandas as pd

import geopandas as gpd

# Geocoding Module.
import pgeocode

# Map plotting module.
import folium
from folium.plugins import MarkerCluster

import json

In [9]:
# Read the data in and set the geocoder to USA.
df = pd.read_excel("/Users/itunuolaitan/Desktop/Portfolio/Code/Data/Folium/us-cities-demographics.xlsx")
nomi = pgeocode.Nominatim('us')

In [10]:
df

,CITY,State,Median Age,Male Population,Female Population,Total Population,Average Household Size,State Code,Race,Count
0,Denver,Colorado,34.1,341137,341408,682545,2.33,CO,Black or African-American,72288
1,Provo,Utah,23.6,56231,59027,115258,3.28,UT,American Indian and Alaska Native,1916
2,Hampton,Virginia,35.5,66214,70240,136454,2.48,VA,Hispanic or Latino,7513
3,Birmingham,Alabama,35.6,102122,112789,214911,2.21,AL,Asian,1500
4,Greeley,Colorado,31.0,50792,50091,100883,2.75,CO,White,92874
...,...,...,...,...,...,...,...,...,...,...
2883,Mesa,Arizona,36.9,234998,236835,471833,2.68,AZ,White,413010
2884,Corpus Christi,Texas,35.0,160488,163594,324082,2.69,TX,Hispanic or Latino,200737
2885,Bismarck,North Dakota,38.0,34675,35565,70240,2.11,ND,Hispanic or Latino,1667
2886,Orem,Utah,26.1,48695,45762,94457,3.26,UT,American Indian and Alaska Native,976


In [12]:
df['Female Population'] = df['Female Population'].astype(int)
df['Male Population'] = df['Male Population'].astype(int)

In [13]:
df_read = df.copy()
# Create latitude and longitude columns from querying the zip code.
#df_read[['Latitude', 'Longitude']] = nomi.query_postal_code(df['ZIP CODE'].to_list())[['latitude', 'longitude']]
# Drop any rows that don't have latitude and longitude values.
#df_read = df_read.dropna(subset=['Longitude', 'Latitude'])

In [15]:
df_read.head(4)

,CITY,State,Median Age,Male Population,Female Population,Total Population,Average Household Size,State Code,Race,Count
0,Denver,Colorado,34.1,341137,341408,682545,2.33,CO,Black or African-American,72288
1,Provo,Utah,23.6,56231,59027,115258,3.28,UT,American Indian and Alaska Native,1916
2,Hampton,Virginia,35.5,66214,70240,136454,2.48,VA,Hispanic or Latino,7513
3,Birmingham,Alabama,35.6,102122,112789,214911,2.21,AL,Asian,1500


In [16]:
city_geo = gpd.read_file("/Users/itunuolaitan/Desktop/Portfolio/Code/Folium/California_Incorporated_Cities_(outline).geojson")
#extract name and geometry
city_geo = city_geo[['CITY', 'geometry']]
city_geo.head(1)

,CITY,geometry
0,Adelanto,"MULTIPOLYGON (((-117.44541 34.66046, -117.4457..."


In [17]:
#merge city geo data and policy data
data = city_geo.merge(df_read,on="CITY")
data.head(6)

,CITY,geometry,State,Median Age,Male Population,Female Population,Total Population,Average Household Size,State Code,Race,Count
0,Alameda,"MULTIPOLYGON (((-122.33016 37.79712, -122.3307...",California,41.4,37747,40867,78614,2.52,CA,Asian,27984
1,Alameda,"MULTIPOLYGON (((-122.33016 37.79712, -122.3307...",California,41.4,37747,40867,78614,2.52,CA,American Indian and Alaska Native,1329
2,Alameda,"MULTIPOLYGON (((-122.33016 37.79712, -122.3307...",California,41.4,37747,40867,78614,2.52,CA,Hispanic or Latino,8265
3,Alameda,"MULTIPOLYGON (((-122.33016 37.79712, -122.3307...",California,41.4,37747,40867,78614,2.52,CA,Black or African-American,7364
4,Alameda,"MULTIPOLYGON (((-122.33016 37.79712, -122.3307...",California,41.4,37747,40867,78614,2.52,CA,White,44232
5,Albany,"POLYGON ((-122.28808 37.89800, -122.28809 37.8...",New York,32.8,47627,50825,98452,2.08,NY,White,58368


In [18]:
# Assigning centroids to map
x_map=data.centroid.x.mean()
y_map=data.centroid.y.mean()
print(x_map,y_map)

-119.56149211601375 35.70295865877037


<ipython-input-18-fa1cee95690c>:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x_map=data.centroid.x.mean()
<ipython-input-18-fa1cee95690c>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y_map=data.centroid.y.mean()


In [22]:
# Set the map and the zoom level and set the cluster to add them to the map.
map = folium.Map(location=[y_map, x_map], zoom_start=5)

folium.Choropleth(
    geo_data=city_geo,
    name="chloropleth",
    data=df_read,
    columns=["CITY", "Total Population"],
    key_on="feature.properties.CITY",
    fill_color="OrRd",
    nan_fill_color="grey",
    fill_opacity=0.7,
    line_opacity=.1,
    legend_name="Population",
).add_to(map)

g = folium.GeoJson(
    data,
    style_function = lambda x: {'fillColor': '#ffffff', 
                            'color':'#000000', 
                            'fillOpacity': 0.1, 
                            'weight': 0.1},
    highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.50, 
                                'weight': 0.1},
    name='geojson'
    ).add_to(map)

folium.GeoJsonTooltip(
    fields=['CITY',
            'Total Population',
            'Female Population',
            'Male Population',
            'Median Age',
            'Average Household Size'],
    aliases=['City:',
            'Total Population:',
            'Female Population:',
            'Male Population:',
            'Median Age:',
            'Average Household Size:'],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
).add_to(g)

folium.LayerControl().add_to(map)



# Save the map.
map.save('Population.html')